In [5]:
"""
COGS 181 Project
Author: Brian Henriquez, Simon Fong, Wilson Tran
"""

'\nCOGS 118B Project\nAuthor: Brian Henriquez, Simon Fong, Wilson Tran\n'

In [6]:
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical
import numpy as np
import os
import cv2
import random
import matplotlib
from datetime import datetime                   # Use to record time
import json                                     # Writing data to logger
matplotlib.use('Agg')                           # Stops from plotting to screen
import matplotlib.pyplot as plt
from dataset import Dataset                     # Custom Dataset

Using TensorFlow backend.


In [44]:
DATASET_NAME = "../../simon/keras/plankton"
IMAGE_WIDTH,IMAGE_HEIGHT,NUM_CHANNELS = 299,299,3
EPOCHS = 100
BATCH_SIZE = 50
NUM_TRAIN,NUM_VAL,NUM_TEST = 20,15,65


ID = "{}_{}_{}_{}_{}_{}_{}".format("5De256relue",DATASET_NAME,
                                EPOCHS,BATCH_SIZE,NUM_TRAIN,NUM_VAL,NUM_TEST)


# Load dataset
cal = Dataset(DATASET_NAME,IMAGE_HEIGHT,IMAGE_WIDTH)
cal.read_data()
cal.train_val_test_split(NUM_TRAIN,NUM_VAL,NUM_TEST)
num_classes = cal.num_classes

Loaded data from ../../simon/keras/plankton
Split 30336 data into 6067 training, 4550 validation, and 19718 testing data.


In [45]:
def load_model():
    """Returns a pretrained model"""
    
    # Loads base model
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS))
    print("Model weights loaded.")
    
    base_out = base_model.output

    # Add more layers
    x = Flatten()(base_out)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # Additional variation 
    x = Dense(256,activation='relu')(x)
    x = Dense(256,activation='relu')(x)
    x = Dense(256,activation='relu')(x)
    x = Dense(256,activation='relu')(x)
    x = Dense(256,activation='relu')(x)

    # Final fully connected layer to work with our data
    predictions = Dense(num_classes,activation='softmax')(x)

    # Build a final model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    print("Model structure")
    model.summary()
    
    # Compile model
    model.compile(optimizers.SGD(lr=1e-4,momentum=0.9),
                'categorical_crossentropy', metrics=['accuracy'])
    print("Model compiled")

    return model

In [46]:
def logger(message):
    """Logs any message into a file"""
    with open('../models/stats.txt', 'a+') as f:
        print >>f, message

In [49]:
def main():
    
    # Make model
    model = load_model()
    print("Model created\n")

    # Init data array to plot
    train_acc = np.array([])
    train_val_acc = np.array([])
    train_loss = np.array([])
    train_val_loss = np.array([])
    
    # Load the training data
    X_train, Y_train = cal.load_training()
    
    # Load the validation data
    X_val, Y_val = cal.load_validation()
    
    # Start time
    start_time = datetime.now()
    print('Start Time', start_time)
    
    # Train model and store stats in history
    history = model.fit(x=X_train,y=Y_train,batch_size=BATCH_SIZE,
                        epochs=EPOCHS,validation_data=(X_val,Y_val))
    
    # End time
    stop_time = datetime.now()
    print('Stop Time', stop_time)
    
    # Print total time
    elapsed_time = stop_time - start_time
    print('My Elapsed Time', elapsed_time)
    logger(elapsed_time)   

    # Append the accuracy and loss scores
    train_acc = np.append(train_acc, history.history['acc'])
    train_val_acc = np.append(train_val_acc, history.history['val_acc'])
    train_loss = np.append(train_loss, history.history['loss'])
    train_val_loss = np.append(train_val_loss, history.history['val_loss'])
    
     
    # Save model weights
    model.save('../models/{}.h5'.format(ID))
    logger(ID)
    logger(history.history)
    print("Model weights saved.")

    
    # Plot accuracy
    plt.hold(True)
    plt.plot(train_acc, label = "Train Accuracy")
    plt.plot(train_val_acc, label = "Validation Accuracy")
    plt.legend(loc='upper right')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig('../plots/acc_vs_val_acc_{}.png'.format(ID))
    plt.hold(False)
    plt.show()

    # Plot loss
    plt.hold(True)
    plt.plot(train_loss, label = "Train Loss")
    plt.plot(train_val_loss, label = "Validation Loss")
    plt.legend(loc='upper right')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig('./plots/loss_vs_val_loss_{}.png'.format(ID))
    plt.hold(False)
    plt.show()
    
    # Test the model
    X_test, Y_test = cal.load_testing()
    metrics = model.evaluate(x=X_test,y=Y_test, batch_size=BATCH_SIZE)
    
    print(metrics)
    logger(metrics)
    print(model.metrics_names)
    logger(model.metrics_names)

    return 0

In [ ]:
if __name__ == '__main__':
    main()

Model weights loaded.
Model structure
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_95 (Conv2D)               (None, 149, 149, 32)  864         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_95 (BatchNor (None, 149, 149, 32)  96          conv2d_95[0][0]                  
____________________________________________________________________________________________________
activation_95 (Activation)       (None, 149, 149, 32)  0           batch_normalization_95[0][0]     
_____________________________________________________

____________________________________________________________________________________________________
conv2d_157 (Conv2D)              (None, 17, 17, 192)   258048      activation_156[0][0]             
____________________________________________________________________________________________________
conv2d_162 (Conv2D)              (None, 17, 17, 192)   258048      activation_161[0][0]             
____________________________________________________________________________________________________
batch_normalization_157 (BatchNo (None, 17, 17, 192)   576         conv2d_157[0][0]                 
____________________________________________________________________________________________________
batch_normalization_162 (BatchNo (None, 17, 17, 192)   576         conv2d_162[0][0]                 
____________________________________________________________________________________________________
activation_157 (Activation)      (None, 17, 17, 192)   0           batch_normalization_157[

Loaded training data 6067/6067   
Loading validation data...
Loaded validation data550/4550   
('Start Time', datetime.datetime(2017, 12, 12, 17, 56, 20, 178551))
Train on 6067 samples, validate on 4550 samples
Epoch 1/100
6067/6067 [==============================] - 102s - loss: 4.8059 - acc: 0.0277 - val_loss: 4.7698 - val_acc: 0.0411
Epoch 2/100
6067/6067 [==============================] - 93s - loss: 4.7041 - acc: 0.0672 - val_loss: 4.6686 - val_acc: 0.0855
Epoch 3/100
6067/6067 [==============================] - 92s - loss: 4.4671 - acc: 0.0814 - val_loss: 4.6226 - val_acc: 0.0382
Epoch 4/100
6067/6067 [==============================] - 92s - loss: 4.2716 - acc: 0.1038 - val_loss: 4.4801 - val_acc: 0.0686
Epoch 5/100
6067/6067 [==============================] - 92s - loss: 4.0983 - acc: 0.1396 - val_loss: 4.2909 - val_acc: 0.1046
Epoch 6/100
6067/6067 [==============================] - 92s - loss: 3.9380 - acc: 0.1567 - val_loss: 3.9467 - val_acc: 0.1429
Epoch 7/100
6067/6067 [===

6067/6067 [==============================] - 90s - loss: 0.1567 - acc: 0.9618 - val_loss: 2.2508 - val_acc: 0.6268
Epoch 64/100
6067/6067 [==============================] - 91s - loss: 0.1511 - acc: 0.9646 - val_loss: 2.3665 - val_acc: 0.6189
Epoch 65/100
6067/6067 [==============================] - 91s - loss: 0.1344 - acc: 0.9710 - val_loss: 2.4542 - val_acc: 0.6138
Epoch 66/100
6067/6067 [==============================] - 91s - loss: 0.1430 - acc: 0.9646 - val_loss: 2.3306 - val_acc: 0.6334
Epoch 67/100
6067/6067 [==============================] - 91s - loss: 0.1339 - acc: 0.9700 - val_loss: 2.3765 - val_acc: 0.6303
Epoch 68/100
6067/6067 [==============================] - 91s - loss: 0.1228 - acc: 0.9721 - val_loss: 2.3368 - val_acc: 0.6251
Epoch 69/100
6067/6067 [==============================] - 90s - loss: 0.1212 - acc: 0.9730 - val_loss: 2.6140 - val_acc: 0.5989
Epoch 70/100
6067/6067 [==============================] - 90s - loss: 0.1199 - acc: 0.9695 - val_loss: 2.4358 - val_a